In [1]:
import palom
import dask.array as da
import tifffile
import ome_types
from unidecode import unidecode

/omics/odcf/analysis/OE0536_projects_temp/spatialomicsdata/MICHELE/COMET_FIX/aligner/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Based on the example at: https://github.com/labsyspharm/palom?tab=readme-ov-file#for-tiff-and-ome-tiff-files

In [2]:
path1 = "" # First tiff file, the run metadata are read from here
path2 = "" # Second tiff file

In [3]:
c1r = palom.reader.OmePyramidReader(path1)
c2r = palom.reader.OmePyramidReader(path2)

In [4]:
# `LEVEL = 0` for processing lowest level pyramid (full resolution)
LEVEL = 0
# choose a higher pyramid level for feature-based affine registration as
# initial coarse alignment
THUMBNAIL_LEVEL = c1r.get_thumbnail_level_of_size(1000)

In [5]:
c21l = palom.align.Aligner(
    ref_img=c1r.read_level_channels(LEVEL, 0),
    moving_img=c2r.read_level_channels(LEVEL, 0),
    ref_thumbnail=c1r.read_level_channels(THUMBNAIL_LEVEL, 0).compute(),
    moving_thumbnail=c2r.read_level_channels(THUMBNAIL_LEVEL, 1).compute(),
    ref_thumbnail_down_factor=c1r.level_downsamples[THUMBNAIL_LEVEL] / c1r.level_downsamples[LEVEL],
    moving_thumbnail_down_factor=c2r.level_downsamples[THUMBNAIL_LEVEL] / c2r.level_downsamples[LEVEL]
)

In [6]:
# run feature-based affine registration using thumbnails
c21l.coarse_register_affine(n_keypoints=4000, plot_match_result=False)
# after coarsly affine registered, run phase correlation on each of the
# corresponding chunks (blocks/pieces) to refine translations
c21l.compute_shifts()
# discard incorrect shifts which is usually due to low contrast in the
# background regions; this is needed for WSI but maybe not for ROI images
c21l.constrain_shifts()

2025-08-04 13:53:37.299 | DEBUG    | palom.register:cv2_feature_detect_and_match:288 - keypts L:4000, keypts R:4000
2025-08-04 13:53:37.634 | DEBUG    | palom.register:cv2_feature_detect_and_match:288 - keypts L:3953, keypts R:4000
2025-08-04 13:53:37.939 | DEBUG    | palom.register:cv2_feature_detect_and_match:288 - keypts L:4000, keypts R:4000
2025-08-04 13:53:38.244 | DEBUG    | palom.register:cv2_feature_detect_and_match:288 - keypts L:3188, keypts R:3235
2025-08-04 13:53:38.421 | INFO     | palom.align:compute_shifts:203 - Computing block-wise shifts
Computing shifts: 100%|##########| 27786/27786 [01:25<00:00, 324.80it/s] 


In [7]:
# configure the transformation of aligning the moving image to the registration
# reference
c2m = palom.align.block_affine_transformed_moving_img(
    ref_img=c1r.read_level_channels(LEVEL, 0),
    # select all the three channels (RGB) in moving image to transform
    moving_img=c2r.pyramid[LEVEL],
    mxs=c21l.block_affine_matrices_da
)

In [8]:
c2m.shape

(5, 31946, 31953)

In [9]:
c1r.pyramid[LEVEL].shape

(9, 31946, 31953)

In [10]:
# Concatenate the registered image to the first image (full resolution)
img = da.concatenate([c1r.pyramid[LEVEL], c2m], axis=0)
img.shape

(14, 31946, 31953)

In [11]:
# Write out the OME-TIFF

output_path = ""

palom.pyramid.write_pyramid(
    mosaics=[img],
    output_path=output_path,
    pixel_size=c1r.pixel_size*c1r.level_downsamples[LEVEL],
    save_RAM=False
)

2025-08-04 13:55:56.959 | INFO     | palom.reader:pixel_size:156 - Detected pixel size: 0.2800 µm
2025-08-04 13:55:56.961 | INFO     | palom.pyramid:write_pyramid:170 - Writing to OUTPUT/COVIMAX-001.ome.tif
Assembling mosaic  1/ 1 (channel 14/14): 100%|##########| 15879/15879 [00:36<00:00, 430.41it/s] 
2025-08-04 14:04:26.104 | INFO     | palom.pyramid:write_pyramid:197 - Generating pyramid
2025-08-04 14:04:26.107 | INFO     | palom.pyramid:write_pyramid:200 -     Level 1 (15973 x 15977)
Processing channel: 100%|##########| 14/14 [02:46<00:00, 11.89s/it]
2025-08-04 14:07:12.664 | INFO     | palom.pyramid:write_pyramid:200 -     Level 2 (7987 x 7989)
Processing channel: 100%|##########| 14/14 [00:42<00:00,  3.06s/it]
2025-08-04 14:07:56.635 | INFO     | palom.pyramid:write_pyramid:200 -     Level 3 (3994 x 3995)
Processing channel: 100%|##########| 14/14 [00:17<00:00,  1.22s/it]
2025-08-04 14:08:14.125 | INFO     | palom.pyramid:write_pyramid:200 -     Level 4 (1997 x 1998)
Processing c

In [12]:
# Reads the meatada from the first tiff file, these will be copied to the registered OME-TIFF
with tifffile.TiffFile(path1) as tif:
    ome_xml1 = tif.ome_metadata
ome1 = ome_types.from_xml(ome_xml1)

In [13]:
# Reads the meatada from the second tiff file, these will be used to get channel and cycle metadata
with tifffile.TiffFile(path2) as tif:
    ome_xml2 = tif.ome_metadata
ome2 = ome_types.from_xml(ome_xml2)

In [14]:
## Add channels from the other image to the first
c1 = c1r.pyramid[LEVEL].shape[0]
c2 = c2m.shape[0]
for c in range(c2):
    ome2.images[0].pixels.channels[c].id = f"Channel:{c + c1}"
    ome2.images[0].pixels.planes[c].the_c = c + c1 # start from the end of the first image
ome1.images[0].pixels.channels = ome1.images[0].pixels.channels + ome2.images[0].pixels.channels
ome1.images[0].pixels.planes = ome1.images[0].pixels.planes + ome2.images[0].pixels.planes
ome1.images[0].pixels.size_c = c1 + c2
ome1.images[0].pixels.planes

[Plane(
    the_z=0,
    the_t=0,
    the_c=0,
    exposure_time=40.0,
    exposure_time_unit='ms',
 ),
 Plane(
    the_z=0,
    the_t=0,
    the_c=1,
    exposure_time=250.0,
    exposure_time_unit='ms',
 ),
 Plane(
    the_z=0,
    the_t=0,
    the_c=2,
    exposure_time=400.0,
    exposure_time_unit='ms',
 ),
 Plane(
    the_z=0,
    the_t=0,
    the_c=3,
    exposure_time=250.0,
    exposure_time_unit='ms',
 ),
 Plane(
    the_z=0,
    the_t=0,
    the_c=4,
    exposure_time=400.0,
    exposure_time_unit='ms',
 ),
 Plane(
    the_z=0,
    the_t=0,
    the_c=5,
    exposure_time=250.0,
    exposure_time_unit='ms',
 ),
 Plane(
    the_z=0,
    the_t=0,
    the_c=6,
    exposure_time=400.0,
    exposure_time_unit='ms',
 ),
 Plane(
    the_z=0,
    the_t=0,
    the_c=7,
    exposure_time=250.0,
    exposure_time_unit='ms',
 ),
 Plane(
    the_z=0,
    the_t=0,
    the_c=8,
    exposure_time=400.0,
    exposure_time_unit='ms',
 ),
 Plane(
    the_z=0,
    the_t=0,
    the_c=9,
    expos

In [15]:
## Add cycle metadata from the second image to the COMET Horizon, structured annotation 
c1 = ome1.structured_annotations[0].value.any_elements[0].children
c2 = ome2.structured_annotations[0].value.any_elements[0].children
cycleid = int(c1[-1].attributes["CycleID"]) + 1
channel = int(c1[-1].attributes["ID"].split(":")[-1]) + 1
for c in c2:
    c.attributes["CycleID"] = str(int(c.attributes["CycleID"]) + cycleid)
    c.attributes["ID"] = "Channel:"+ str(int(c.attributes["ID"].split(":")[-1]) + channel)
ome1.structured_annotations[0].value.any_elements[0].children = c1 + c2
ome1.structured_annotations[0].value.any_elements[0].children

[AnyElement(qname='{http://www.openmicroscopy.org/Schemas/OME/2016-06}ChannelPriv', text='', tail=None, children=[], attributes={'ID': 'Channel:0', 'CycleID': '0', 'LedCurrent': '20', 'LedCurrentUnit': 'mA', 'SensorGain': '0', 'FluorescenceChannel': 'DAPI'}),
 AnyElement(qname='{http://www.openmicroscopy.org/Schemas/OME/2016-06}ChannelPriv', text='', tail=None, children=[], attributes={'ID': 'Channel:1', 'CycleID': '0', 'LedCurrent': '1700', 'LedCurrentUnit': 'mA', 'SensorGain': '0', 'FluorescenceChannel': 'TRITC'}),
 AnyElement(qname='{http://www.openmicroscopy.org/Schemas/OME/2016-06}ChannelPriv', text='', tail=None, children=[], attributes={'ID': 'Channel:2', 'CycleID': '0', 'LedCurrent': '850', 'LedCurrentUnit': 'mA', 'SensorGain': '0', 'FluorescenceChannel': 'Cy5'}),
 AnyElement(qname='{http://www.openmicroscopy.org/Schemas/OME/2016-06}ChannelPriv', text='', tail=None, children=[], attributes={'ID': 'Channel:3', 'CycleID': '1', 'LedCurrent': '1700', 'LedCurrentUnit': 'mA', 'Sensor

In [16]:
# Convert back to XML and represent unicode as ASCII
ome_xml = ome1.to_xml()
ome_xml = unidecode(ome_xml)

In [17]:
# Write the new XML metadata to the output file (overwrites)
tifffile.tiffcomment(output_path, ome_xml)